<a href="https://colab.research.google.com/github/mark-morelos/DS-Unit-2-Kaggle-Challenge/blob/master/Copy_of_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [87]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [89]:
import numpy as np

# split train into train and val
train, val = train_test_split(train, train_size=0.85, test_size=0.15,
                              stratify=train['status_group'], random_state=42)

# define function to wrangle train, validate, and test sets in the same way
# clean outliers and engineer features
def wrangle(X):
  """Wrangle train, validate, and test sets in the same way"""

  # prevent SettingWithCopyWarning
  X = X.copy()

  # latitude values near 0 = 0
  # remove null island
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # consider 0s at columns that do not belong as null, replace with 0 with null
  # impute missing values; crate column for 'missing indicator'
  cols_with_zeros = ['longitude', 'latitude', 'construction_year',
                     'gps_height', 'population']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)
    X[col + '_MISSING'] =X[col].isnull()
  
  # drop duplicate columns
  duplicates = ['quantity_group', 'payment_type']
  X = X.drop(columns=duplicates)

  # drop recorded_by (never varies) and id (always varies, random)
  unusable_variance = ['recorded_by', 'id']
  X = X.drop(columns=unusable_variance)

  # convert date_recorded to datetime
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

  # extract year/month/date from date_recorded, drop date_recorded
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns='date_recorded')

  # engineer feature: difference of construction_year to year_recorded
  X['years'] = X['year_recorded'] - X['construction_year']
  X['years_MISSING'] = X['years'].isnull()




  return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [90]:
# check data
train.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,longitude_MISSING,latitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING,year_recorded,month_recorded,day_recorded,years,years_MISSING
26290,0.0,NaN,NaN,NaN,34.108531,-8.787551,Kwa Mzee Ng'Wandu,0,Rufiji,Mahelele,Mbeya,12,7,Mbarali,Chimala,NaN,True,VWC,NaN,True,NaN,swn 80,swn 80,handpump,vwc,user-group,pay monthly,soft,good,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional,False,False,True,True,True,2011,4,12,NaN,True
59373,5000.0,Rudep,1137.0,Distri,32.125024,-6.784358,Kwa Kadiega,0,Lake Rukwa,Mapambano,Rukwa,15,1,Mpanda,Ilela,96.0,True,VWC,NaN,True,1988.0,india mark ii,india mark ii,handpump,vwc,user-group,pay annually,soft,good,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional,False,False,False,False,False,2013,1,30,25.0,False
21359,50.0,Ministry Of Water,1173.0,Idara ya maji,34.942900,-6.244980,Kwa Thomas Nyakunya,0,Rufiji,Warenchenza,Singida,13,3,Manyoni,Nkoko,250.0,True,VWC,Mpola water supply,False,1972.0,mono,mono,motorpump,vwc,user-group,pay per bucket,soft,good,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,False,False,False,False,False,2013,1,31,41.0,False
23742,0.0,Wananchi,NaN,wananchi,33.707635,-9.567718,Morovian Church,0,Lake Nyasa,Kandete,Mbeya,12,3,Ileje,Ikinga,NaN,True,Water authority,Ngana water supplied scheme,True,NaN,gravity,gravity,gravity,water authority,commercial,pay monthly,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,False,False,True,True,True,2011,7,21,NaN,True
43676,0.0,World Vision,NaN,World vision,33.926652,-3.487160,Mwakidiga Primary School,0,Internal,Dakama,Shinyanga,17,2,Maswa,Lalago,NaN,True,WUG,NaN,False,NaN,gravity,gravity,gravity,wug,user-group,other,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe multiple,communal standpipe,functional needs repair,False,False,True,True,True,2012,11,10,NaN,True


In [91]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [92]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [93]:
# use a model for classification - RandomForestClassifier

import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(random_state=0, n_jobs=-1)
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.8085297418630752


In [94]:
# rewrite so ALL features are in training set and add hyperparameters
from scipy.stats import randint, uniform

X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(random_state=0, n_jobs=-1)

)

param_distributions = {
    'randomforestclassifier__max_depth':[5, 10, 15, 20, None],
    'randomforestclassifier__n_estimators': randint(30, 70),
    'randomforestclassifier__max_features': uniform(0,1)
}

# fit on train, score on val
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.8122334455667789


In [95]:
# RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=5,
    cv=3,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1

)

search.fit(X_train, y_train);

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.9min finished


In [96]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.23928625498767397, 'randomforestclassifier__n_estimators': 67}
Cross-validation accuracy 0.8035848682907507


In [97]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,4,2,0,3,1
mean_fit_time,11.626,16.8352,6.04743,17.6729,9.58524
std_fit_time,2.13664,0.275032,0.0225596,0.140864,0.247005
mean_score_time,0.758122,0.883673,0.608532,0.735561,0.591604
std_score_time,0.25224,0.135895,0.0974473,0.0905254,0.1149
param_randomforestclassifier__max_depth,20,20,20,15,5
param_randomforestclassifier__max_features,0.239286,0.409029,0.172522,0.769843,0.544754
param_randomforestclassifier__n_estimators,67,51,33,33,59
params,"{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand..."
split0_test_score,0.801248,0.800357,0.798574,0.791444,0.716102
split1_test_score,0.806061,0.805526,0.805407,0.79798,0.723232


In [98]:
y_pred = pipeline.predict(X_test)
y_pred

array(['functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [99]:
DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)

In [100]:
from google.colab import files
files.download('your-submission-filename.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>